In [30]:
import plotly.express as px

import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.distributions.empirical_distribution import ECDF

from scipy.optimize import minimize
import matplotlib.pyplot as plt

import seaborn as sns



import plotly.express as px
from plotly import io, offline # io to save a html-file of the animation
offline.init_notebook_mode() # offline for better looking interactive graphics
import plotly.figure_factory as ff

import plotly.graph_objects as go  #plotly 4.0.0rc1

import plotly.io as pio
# Set the default renderer to 'colab'
pio.renderers.default = 'colab'

np.random.seed(1)

In [7]:
markets_data = pd.read_csv('markt_new.csv', delimiter=';')

# Display the first few rows of the dataframe and the column names
markets_data.head(), markets_data.columns


(   Column1 Region Standort       Monat  Kunden    Umsatz Früchte und Gemüse  \
 0        1     BE     Bern  01.01.2022    3632   50954,6            5689,55   
 1        2     BE     Bern  01.02.2022    3586  48594,55            6523,98   
 2        3     BE     Bern  01.03.2022    4720   70197,9            8400,48   
 3        4     BE     Bern  01.04.2022    4962   74841,1           11515,46   
 4        5     BE     Bern  01.05.2022    5383   79644,5           11685,18   
 
   Getränke Haltbare Grundnahrungsmittel  \
 0   1734,2                      6969,95   
 1  3219,55                     12235,05   
 2   4734,5                     10032,25   
 3   3913,5                     12576,15   
 4   5298,7                      16417,5   
 
   Kühl-,Molkerei- und Charcuterieprodukte Material Near Food Non Food  \
 0                                 6771,13      110   4382,15   1108,9   
 1                               4825,8375    229,6   3716,45   1628,2   
 2                            

In [6]:
# Convert 'Monat' to datetime and extract the year
markt_new_data['Year'] = pd.to_datetime(markt_new_data['Monat'], format='%d.%m.%Y').dt.year

# Group by 'Standort' and 'Year', and sum the 'Umsatz'
umsatz_per_standort_year = markt_new_data.groupby(['Standort', 'Year'])['Umsatz'].sum().reset_index()

# Preview the grouped data
umsatz_per_standort_year.head()

NameError: name 'markt_new_data' is not defined

In [ ]:
# Convert 'Monat' to datetime and extract the year
markets_data['Year'] = pd.to_datetime(markets_data['Monat'], format='%d.%m.%Y').dt.year

# Group by 'Standort' and 'Year', and sum the 'Umsatz'
umsatz_per_standort_year = markt_new_data.groupby(['Standort', 'Year'])['Umsatz'].sum().reset_index()

# Preview the grouped data
umsatz_per_standort_year.head()

,Standort,Year,Umsatz
0,Baar,2022,"2423722819,1532532,131426,6531645,4533287,2288..."
1,Baar,2023,"37538,232379,138878,534492,5537211,137213,2326..."
2,Baar,2024,"31604,0530983,25"
3,Basel,2022,"39970,236720,5548830,1558482,661222,5553588,44..."
4,Basel,2023,"52149,4545560,5558439,0549529,9551258,3548055,..."


In [19]:
markets_data

print(markets_data.head())
print(markets_data['Monat'].dtype)
print(markets_data.info())

# Convert 'Monat' to datetime
markets_data['Monat'] = pd.to_datetime(markets_data['Monat'], format='%d.%m.%Y', errors='coerce')

#Extract the Year
markets_data['Year'] = markets_data['Monat'].dt.year

#Verifying a few Monat and Year rows
print(markets_data[['Monat', 'Year']].head())

print(markets_data[markets_data['Monat'].isna()])

Empty DataFrame
Columns: [Column1, Region, Standort, Monat, Kunden, Umsatz, Früchte und Gemüse, Getränke, Haltbare Grundnahrungsmittel, Kühl-,Molkerei- und Charcuterieprodukte, Material, Near Food, Non Food, Subventionierte Grundnahrungsmittel, Süssigkeiten und Snacks, Tiefkühlprodukte, Year]
Index: []


In [14]:
case_data = pd.read_csv('Falldaten_CaritasbB_updated.csv', delimiter=';')

print(case_data.head())
print(case_data.info())
#
case_data['Datum_Leistung'] = pd.to_datetime(case_data['Datum_Leistung'], errors='coerce')
case_data['Datum_Registrierung_Client'] = pd.to_datetime(case_data['Datum_Registrierung_Client'], errors='coerce')

# Extract the Year
case_data['Leistung_Year'] = case_data['Datum_Leistung'].dt.year
case_data['Registrierung'] = case_data['Datum_Registrierung_Client'].dt.year

# Verify the new columns
print(case_data[['Leistung_Year', 'Registrierung']].head())




   Unnamed: 0 Datum_Leistung    Ereignis_EreignisartID Ereignis_Anzahl  \
0           1     2024-04-19                   Telefon               5   
1           2     2024-04-19                   Telefon               5   
2           3     2024-04-18  Besprechung mit KlientIn              30   
3           4     2024-04-18  Besprechung mit KlientIn              10   
4           5     2024-04-18                   Telefon               5   

   Ereignis_Einheit_ID Datum_Registrierung_Client         Hauptart  \
0                    2                 2019-01-17  02 Finanzbedarf   
1                    2                 2019-01-18  02 Finanzbedarf   
2                    2                 2022-01-12         04 Recht   
3                    2                 2022-02-02      01 Soziales   
4                    2                 2023-09-11  02 Finanzbedarf   

               Materielle_G                         Kulturelle_G  \
0  01 zu geringes Einkommen                             00 keine  

In [32]:

#CODE FOR POLAR GRAPHS OF HAUPTART BY YEAR
# Filter the years first to ensure only 2022, 2023, and 2024 are included
filtered_data = case_data[case_data['Leistung_Year'].isin([2022, 2023, 2024])]

# Group by 'Leistung_Year' and 'Hauptart' and count occurrences
category_counts = filtered_data.groupby(['Leistung_Year', 'Hauptart']).size().reset_index(name='Counts')

filtered_data.info()


# Create a polar bar chart
fig = px.bar_polar(category_counts, r="Counts", theta="Hauptart",
                   color="Leistung_Year", template="plotly",
                   color_discrete_sequence=px.colors.sequential.Plasma_r,
                   title="Category Distribution by Year")
# Show the plot
fig.show()

#Customization
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, category_counts['Counts'].max()]
        )
    ),
    title={'text': "Category Distribution by Year", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}
)

fig.show()


<class 'pandas.core.frame.DataFrame'>
Index: 9203 entries, 0 to 9202
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unnamed: 0                  9203 non-null   int64         
 1   Datum_Leistung              9203 non-null   datetime64[ns]
 2   Ereignis_EreignisartID      9203 non-null   object        
 3   Ereignis_Anzahl             9203 non-null   object        
 4   Ereignis_Einheit_ID         9203 non-null   int64         
 5   Datum_Registrierung_Client  9202 non-null   datetime64[ns]
 6   Hauptart                    9203 non-null   object        
 7   Materielle_G                9203 non-null   object        
 8   Kulturelle_G                9203 non-null   object        
 9   Soziale_G                   9203 non-null   object        
 10  Gesundheitliche_G           9203 non-null   object        
 11  Rechtliche_G                9203 non-null   object        
 1

In [36]:
#CODE FOR BARGRAPHS OF HAUPTART BY YEAR

# Aggregate the counts of each category by year
category_counts = case_data[case_data['Leistung_Year'].isin([2022, 2023, 2024])] \
                    .groupby(['Leistung_Year', 'Hauptart']).size() \
                    .reset_index(name='Counts')

# Create the bar graph
fig = px.bar(category_counts,
             x='Hauptart',
             y='Counts',
             color='Leistung_Year',
             title='Category Distribution by Year',
             labels={'Counts': 'Number of Occurrences', 'Hauptart': 'Category'},
             barmode='group')

# Show the plot
fig.show()


# Initialize a figure
fig = go.Figure()

# Retrieve unique years and categories
years = sorted(category_counts['Leistung_Year'].unique())
categories = category_counts['Hauptart'].unique()

# Create a bar for each year
for year in years:
    filtered_data = category_counts[category_counts['Leistung_Year'] == year]
    fig.add_trace(go.Bar(
        x=filtered_data['Hauptart'],
        y=filtered_data['Counts'],
        name=str(year)  # Name the bars as per the year
    ))

# Update the layout for grouped bars
fig.update_layout(
    barmode='group',  # This makes bars appear next to each other instead of stacking
    title='Category Distribution by Year',
    xaxis_title='Category',
    yaxis_title='Count',
    legend_title='Year',
    plot_bgcolor='white',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

# Show the plot
fig.show()
